# Importing modules

In [4]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm
import numpy as np
import string

dir_path = os.getcwd()

In [5]:
base_url = 'https://www.youtube.com'
df_url = pd.read_csv("urls.csv")
df_url.head()

,url,id,title
0,/watch?v=0vFTfoOavRM&list=UUt7fwAhXDy3oNFTAzF2...,0vFTfoOavRM,Perfume Genius - Set My Heart on Fire Immediat...
1,/watch?v=nwJ88RCBcfE&list=UUt7fwAhXDy3oNFTAzF2...,nwJ88RCBcfE,Moses Sumney - græ ALBUM REVIEW
2,/watch?v=4uNlwM0ZfY8&list=UUt7fwAhXDy3oNFTAzF2...,4uNlwM0ZfY8,Yung Lean - Starz ALBUM REVIEW
3,/watch?v=HRZcBNjlPQI&list=UUt7fwAhXDy3oNFTAzF2...,HRZcBNjlPQI,Charli XCX - how i'm feeling now ALBUM REVIEW
4,/watch?v=LqBsfpdHXX0&list=UUt7fwAhXDy3oNFTAzF2...,LqBsfpdHXX0,Future - High Off Life ALBUM REVIEW


# Collecting transcripts

In [6]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import CouldNotRetrieveTranscript, NoTranscriptAvailable, TranscriptsDisabled
errors = (CouldNotRetrieveTranscript, NoTranscriptAvailable, TranscriptsDisabled)

In [11]:
df_transcripts = pd.DataFrame(columns = ["id", "full_transcript"])
df_transcripts.id = df_url.id

In [ ]:
pbar = tqdm(total = len(df_url.id))
breaks = 0

for count, ids in enumerate(df_url.id):
    try:
        full_text = ""
        for line in YouTubeTranscriptApi.get_transcript(ids):
            full_text += line["text"] + " "
        df_transcripts.full_transcript[count] = full_text
    except errors:
        df_transcripts.full_transcript[count] = ""
    pbar.update()
    if count % 100 == 99:
        df_transcripts.to_csv("transcripts/transcripts_{}.csv".format(breaks), index = False)
        df_transcripts = pd.DataFrame(columns = ["id", "full_transcript"])
        df_transcripts.id = df_url.id
        breaks += 1
        
pbar.close()
df_transcripts.head()

# Merging all transcripts

In [18]:
from os import listdir, makedirs
from os.path import isfile, join, exists

if not exists("transcripts"):
    makedirs("transcripts")

transcript_path = join(dir_path, "transcripts")
transcripts = [pd.read_csv(join(transcript_path, f)) for f in listdir(transcript_path) if isfile(join(transcript_path, f))]
df_transcripts = transcripts[0]
pbar = tqdm(total = len(transcripts))
for i, transcript in enumerate(transcripts):
    if i == 0:
        pbar.update()
        continue
    #df_transcripts = pd.merge(df_transcripts, transcript, left_index = True, right_index = True, on="id")
    df_transcripts = df_transcripts.append(transcript, sort = False)
    pbar.update()
pbar.close()

<ipython-input-18-a6d51f39dd97>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm(total = len(transcripts))


In [20]:
df_transcripts.to_csv("full_transcripts.csv", index = False, sep = ";")